In [5]:
import pandas as pd
import random

# 1) Combine Bloom files
df_bloom_p1 = pd.read_csv("graded_biographies_part1_bloom.csv")
df_bloom_p2 = pd.read_csv("graded_biographies_part2_bloom.csv")
# If there's a part3 for Bloom, concatenate it too:
# df_bloom_p3 = pd.read_csv("graded_biographies_part3_bloom.csv")
# df_bloom = pd.concat([df_bloom_p1, df_bloom_p2, df_bloom_p3], ignore_index=True)
df_bloom = pd.concat([df_bloom_p1, df_bloom_p2], ignore_index=True)

# Rename columns so they won’t conflict when merging
df_bloom = df_bloom.rename(columns={
    "summary": "bloom_summary",
    "grade":   "bloom_grade"
})

# 2) Combine LLaMA-2 files
df_llama_p1 = pd.read_csv("graded_biographies_part1_llama2.csv")
df_llama_p2 = pd.read_csv("graded_biographies_part2_llama2.csv")
df_llama_p3 = pd.read_csv("graded_biographies_part3_llama2.csv")
df_llama = pd.concat([df_llama_p1, df_llama_p2, df_llama_p3], ignore_index=True)

df_llama = df_llama.rename(columns={
    "summary": "llama_summary",
    "grade":   "llama_grade"
})

# 3) Load the test-summaries CSVs for GPT-Neo, Phi, and TinyLlama
df_neo  = pd.read_csv("graded_test_summaries_GPT-Neo-125M.csv").rename(
    columns={"generated_summary": "neo_summary", "grade": "neo_grade"}
)
df_phi  = pd.read_csv("graded_test_summaries_Phi-1.5B.csv").rename(
    columns={"generated_summary": "phi_summary", "grade": "phi_grade"}
)
df_tiny = pd.read_csv("graded_test_summaries_TinyLlama-1.1B.csv").rename(
    columns={"generated_summary": "tiny_summary", "grade": "tiny_grade"}
)

# 4) Merge Bloom and LLaMA-2 on "document"
df_bloom_llama = df_bloom.merge(df_llama, on="document", how="inner")

# 5) Merge with GPT-Neo
df_merged = df_bloom_llama.merge(df_neo, on="document", how="left")
# 6) Merge with Phi
df_merged = df_merged.merge(df_phi, on="document", how="left")
# 7) Merge with TinyLlama
df_merged = df_merged.merge(df_tiny, on="document", how="left")

# Now df_merged has columns:
#   ["document", "bloom_summary", "bloom_grade", 
#    "llama_summary", "llama_grade", 
#    "neo_summary", "neo_grade",
#    "phi_summary", "phi_grade",
#    "tiny_summary", "tiny_grade"]

# 8) Pick a random row to compare
row = df_merged.sample(n=1).iloc[0]

# 9) Build a small LaTeX table snippet
latex_table = f"""
\\begin{{table}}[ht!]
\\centering
\\begin{{tabular}}{{|p{{12cm}}|}}
\\hline
\\textbf{{Excerpt from Document}}: {row['document'][:300]}... \\\\ \\hline
\\textbf{{Bloom Summary}}: {row['bloom_summary']} \\\\ \\hline
\\textbf{{LLaMA-2 Summary}}: {row['llama_summary']} \\\\ \\hline
\\textbf{{GPT-Neo-125M Summary}}: {row['neo_summary']} \\\\ \\hline
\\textbf{{Phi-1.5B Summary}}: {row['phi_summary']} \\\\ \\hline
\\textbf{{TinyLlama-1.1B Summary}}: {row['tiny_summary']} \\\\ \\hline
\\end{{tabular}}
\\caption{{Comparison of Summaries for a Random Document}}
\\end{{table}}
"""

print(latex_table)



\begin{table}[ht!]
\centering
\begin{tabular}{|p{12cm}|}
\hline
\textbf{Excerpt from Document}: Ahmed Saif Zaal Abu Muhair es un deportista emiratí que compitió en atletismo adaptado. Ganó dos medallas en los Juegos Paralímpicos de Sídney 2000.​... \\ \hline
\textbf{Bloom Summary}: Ahmed Saif Zaal Abu Muhair es un deportista emiratí que compitió en atletismo adaptado. Ganó dos medallas en los Juegos Paralímpicos de Sídney 2000.​
Shoha Al Qassimi es una atleta de atletismo con discapacidad visual. Fue la primera atleta con discapacidad visual en el mundo en competir en una carrera de 5.000 metros.​
La ciudad de Dubái es el hogar de varios equipos deportivos profesionales. El equipo de fútbol de Dubái FC es el equipo de fútbol más antiguo de la ciudad, fundado en 1962.​ Los Dubái Árabes son el equipo de fútbol masculino más antiguo de Dubái y el equipo nacional de fútbol de Dubái. El equipo de fútbol de Dubái femenino es el primer equipo femenino de fútbol de la ciudad y juega en la Lig